<a href="https://colab.research.google.com/github/Ananassio/Big-Data-Analytics/blob/main/week_5/Assigment_NY_Taxi_on_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment: NY Taxi Fare Prediction with DASK

* Note: see [NY Taxi Usecase Notebook](https://colab.research.google.com/github/keuperj/DataEngineering22/blob/main/week_8/UseCase_NY_Taxi.ipynb) for an discription of a non-parallel solution

* NY Taxi Fare Prediction Task + Data: https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/overview

In [1]:
#install DASK
!pip install distributed "dask[complete]" dask-ml graphviz  --ignore-installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 837 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 45.4 MB/s 
     |████████████████████████████████| 148 kB 60.3 MB/s 
     |████████████████████████████████| 46 kB 3.4 MB/s 
     |████████████████████████████████| 96 kB 5.6 MB/s 
     |████████████████████████████████| 428 kB 43.4 MB/s 
     |████████████████████████████████| 40 kB 5.8 MB/s 
     |████████████████████████████████| 1.2 MB 56.0 MB/s 
     |████████████████████████████████| 133 kB 69.5 MB/s 
     |████████████████████████████████| 596 kB 43.7 MB/s 
     |████████████████████████████████| 299 kB 21.2 MB/s 
     |████████████████████████████████| 281 kB 46.9 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 140 kB 51.4 MB/s 
     |████████████████████████████████| 98 kB 2.3 MB/s 
     |████████████████████████████████| 15

In [1]:
#in colab, we need to clone the data from the repo
!git clone https://github.com/keuperj/DATA.git
path='DATA'

fatal: destination path 'DATA' already exists and is not an empty directory.


In [2]:
#read data to Pandas DF
import pandas as pd
data=pd.read_csv("DATA/NY_taxi_train_small.csv")
y=data['fare_amount']
X=data.drop(['fare_amount'],axis=1)


In [3]:
X.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2014-03-12 17:04:30.0000001,2014-03-12 17:04:30 UTC,-73.956721,40.767081,-73.989080,40.772745,1
1,2009-04-17 21:59:00.00000044,2009-04-17 21:59:00 UTC,-73.870913,40.773722,-73.996285,40.716018,2
2,2009-10-06 13:42:00.00000015,2009-10-06 13:42:00 UTC,-73.976258,40.751410,-73.984795,40.751305,3
3,2012-05-02 21:38:39.0000004,2012-05-02 21:38:39 UTC,-73.977940,40.752586,-73.976525,40.667005,1
4,2011-04-21 18:11:13.0000001,2011-04-21 18:11:13 UTC,-73.988390,40.723152,-73.984367,40.736301,1


In [4]:
y.head()

0    11.0
1    23.7
2     5.7
3    28.1
4     5.7
Name: fare_amount, dtype: float64

## Assignment:
use *DASK Dataframes* and *DASK-ML* to 
* split in train and test data
* build a pre-processing,
* feature extraction and 
* prediction pipeline 

to predict the taxi fares. 

### Use
* local DASK cluster
* [DASK DataFrames](https://examples.dask.org/dataframe.html)
* [DASK-ML](https://ml.dask.org/)

### Hints:
* start with a very simple, but working predicten and then enhance the solution with better pre-processing and features

In [5]:
# Cluster setup
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

In [6]:
#get client info
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 12.68 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44251,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 12.68 GiB
Comm: tcp://127.0.0.1:34537,Total threads: 1
Dashboard: http://127.0.0.1:33407/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:46035,


In [7]:
import dask
import dask.dataframe as dd
import dask.array as da
import numpy as np

### Export DASk Dashboard to public URL

In [8]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 3.212s


In [9]:
!lt --port 8787


^C


In [10]:
Xd = dask.dataframe.from_pandas(X.drop(['pickup_datetime','key'],axis=1), npartitions=10)
Xd

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
npartitions=10,,,,,
0,float64,float64,float64,float64,int64
100000,...,...,...,...,...
...,...,...,...,...,...
900000,...,...,...,...,...
999999,...,...,...,...,...


In [11]:
Xd.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,-73.956721,40.767081,-73.989080,40.772745,1
1,-73.870913,40.773722,-73.996285,40.716018,2
2,-73.976258,40.751410,-73.984795,40.751305,3
3,-73.977940,40.752586,-73.976525,40.667005,1
4,-73.988390,40.723152,-73.984367,40.736301,1


In [12]:
from dask_ml.preprocessing import StandardScaler
Xd = StandardScaler().fit_transform(Xd)
Xd.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,-0.116695,0.087540,-0.122419,0.092280,-0.516770
1,-0.109739,0.088228,-0.123019,0.086211,0.238363
2,-0.118279,0.085918,-0.122062,0.089986,0.993496
3,-0.118415,0.086040,-0.121372,0.080969,-0.516770
4,-0.119262,0.082993,-0.122026,0.088381,-0.516770


In [13]:
Xd.replace([np.inf, -np.inf], np.nan)
Xd = Xd.fillna(0)

Xd.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,-0.116695,0.087540,-0.122419,0.092280,-0.516770
1,-0.109739,0.088228,-0.123019,0.086211,0.238363
2,-0.118279,0.085918,-0.122062,0.089986,0.993496
3,-0.118415,0.086040,-0.121372,0.080969,-0.516770
4,-0.119262,0.082993,-0.122026,0.088381,-0.516770


In [14]:
Xd.astype(float)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
npartitions=10,,,,,
0,float64,float64,float64,float64,float64
100000,...,...,...,...,...
...,...,...,...,...,...
900000,...,...,...,...,...
999999,...,...,...,...,...


In [15]:
yd = dask.dataframe.from_pandas(y, npartitions=10)
yd

Dask Series Structure:
npartitions=10
0         float64
100000        ...
           ...   
900000        ...
999999        ...
Name: fare_amount, dtype: float64
Dask Name: from_pandas, 10 tasks

In [16]:
yd.head()

0    11.0
1    23.7
2     5.7
3    28.1
4     5.7
Name: fare_amount, dtype: float64

In [17]:
yd.fillna(0)

Dask Series Structure:
npartitions=10
0         float64
100000        ...
           ...   
900000        ...
999999        ...
Name: fare_amount, dtype: float64
Dask Name: fillna, 20 tasks

In [18]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xd, yd, test_size=0.15)

/usr/local/lib/python3.7/dist-packages/dask_ml/model_selection/_split.py:469: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  category=FutureWarning,


In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
import joblib

In [20]:
RFR = RandomForestRegressor()

In [21]:
%%time
with joblib.parallel_backend('dask'):
    # Your scikit-learn code
    model = RFR.fit(X_train, y_train) 

CPU times: user 54.1 s, sys: 18.3 s, total: 1min 12s
Wall time: 14min 50s


In [27]:
model.score(X_test, y_test)

0.7647415030283491

In [23]:
%%time
with joblib.parallel_backend('dask'):
  Xpred = model.predict(X_test)


CPU times: user 17.9 s, sys: 136 ms, total: 18.1 s
Wall time: 10.1 s


In [24]:
pd.DataFrame(Xpred).head()

,0
0,9.5610
1,4.9330
2,9.1040
3,5.0190
4,23.5177


In [25]:
y_test.head()

0     11.0
12     6.5
22     9.3
25     5.7
27    24.0
Name: fare_amount, dtype: float64

In [26]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test, Xpred))

4.754139716616546